In [1]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
import pandas as pd
from itertools import islice
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [2]:
load_dotenv()
endpoint = os.getenv('AZURE_LANGUAGE_ENDPOINT')
key = os.getenv('AZURE_LANGUAGE_KEY')

textAnalyticsClient = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key=key))

In [3]:
def batch(iterable, batch_size):
    iterator = iter(iterable)
    for first in iterator:
        yield [first, *islice(iterator, batch_size - 1)]

In [ ]:
subset = pd.read_csv('sampleDataForPrediction.csv')

reviews = subset['content'].tolist()

predictedSentiment = []
confidenceScores = []

batchSize = 5
batches = [reviews[i:i + batchSize]for i in range(0, len(reviews), batchSize)]
for batch in batches:
    response = textAnalyticsClient.analyze_sentiment(documents=batch)
    for doc in response:
        predictedSentiment.append(doc.sentiment) 
        confidenceScores.append(
            max(doc.confidence_scores.__dict__.values()) * 100)

subset['AzurePredictedSentiment'] = predictedSentiment
subset['AzureConfidence'] = confidenceScores

In [11]:
subset.head()
subset.to_csv('sampleDataWithPrediction.csv')